In [47]:
import pandas as pd
import os
import re
from ipysheet import from_dataframe, to_dataframe


In [ ]:
# NFCU
path = '/mnt/c/users/robir/Downloads/transactions.CSV'
df = pd.read_csv(path)
df['Date'] = pd.to_datetime(df.Date)
df.drop("No.", axis=1, inplace=True)
df.rename(columns={"Description":"Payee"}, inplace=True)
df.to_csv('/mnt/c/users/robir/Downloads/nfcu.csv')

In [42]:
# Ally
path = '/mnt/c/users/robir/Downloads/transactions.csv'
df = pd.read_csv(path)

df.drop(columns=[' Time', ' Type'], inplace=True)
df.rename(columns={" Description":"Payee"}, inplace=True)

df.to_csv('/mnt/c/users/robir/Downloads/ally.csv')


In [56]:
# Chase
path = '/mnt/c/users/robir/Downloads/'
for filename in os.listdir(path):
    if re.match("Chase.*.CSV",filename):
        path = path + filename
df = pd.read_csv(path)
df.drop(columns=['Transaction Date', 'Category', 'Type'], inplace=True)
df.rename(columns={'Post Date':'Date'}, inplace=True)

df.to_csv('/mnt/c/users/robir/Downloads/Chase.csv')


In [58]:
# Amex
path = '/mnt/c/users/robir/downloads/activity.csv'
df = pd.read_csv(path)

df.rename(columns={"Description":"Payee"}, inplace=True)

df.to_csv('/mnt/c/users/robir/Downloads/amex.csv')